In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification
# Using the dlite function
import torch.nn as nn
from torch.nn import functional as F
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn.metrics import cohen_kappa_score

In [ ]:
!pip install tensorflow gpu

ERROR: Could not find a version that satisfies the requirement gpu (from versions: none)
ERROR: No matching distribution found for gpu


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


# Data Website

Ner dataset website (https://www.kaggle.com/datasets/namanj27/ner-dataset)

# CoNLL-2003(english) Named Entity Recognition (NER)

The  **`CoNLL 2003`** shared task consists of data from the Reuters 1996 news corpus with annotations for 4 types of `Named Entities` (persons, locations, organizations, and miscellaneous entities). The data is in a [IOB2](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging)) format.  Each token enitity has a `'B-'` or `'I-'` tag indicating if it is the start of the entity or if the token is inside the annotation.

* **`Person`**: `'B-PER'` and  `'I-PER'`


* **`Organization`**: `'B-ORG'` and `'I-ORG'`


* **`Location`**: `'B-LOC'`  and `'I-LOC'`


* **`Miscellaneous`**: `'B-MISC'` and `'I-MISC'`


* **`Other(non-named entity)`**: `'O'`



# Reseult

# Original

## Micro Table

| Loss | Train Iteration | Precision | Recall | F1-score | Training time |
| --- | --- | --- | --- | --- | --- |
| CE | 3747 | 0.930 | 0.930 | 0.930 | 3 min 24s |
| DLITE | 3747 | 0.850 | 0.850 | 0.850 | 3 min 24s |
| L1 | 3747 | 0.886 | 0.886 | 0.886 | 3 min 18s |
| L2 | 3747 | 0.930 | 0.930 | 0.930 | 3 min 19s |
| KL | 3747 | 0.959 | 0.959 | 0.959 | 3 min 26s |


## Macro Table

| Loss | Train Iteration | Precision | Recall | F1-score | Training time |
| --- | --- | --- | --- | --- | --- |
| CE | 3747 | 0.705 | 0.661 | 0.663 | 3 min 24s |
| DLITE | 3747 | 0.983 | 0.111 | 0.102 | 3 min 24s |
| L1 | 3747 | 0.619 | 0.271 | 0.312 | 3 min 18s |
| L2 | 3747 | 0.682 | 0.600 | 0.625 | 3 min 19s |
| KL | 3747 | 0.714 | 0.593 | 0.627 | 3 min 26s |

## Weighted Table

| Loss | Train Iteration | Precision | Recall | F1-score | Training time |
| --- | --- | --- | --- | --- | --- |
| CE | 3747 | 0.936 | 0.938 | 0.937 | 3 min 24s |
| DLITE | 3747 | 0.846 | 0.849 | 0.847 | 3 min 24s |
| L1 | 3747 | 0.856 | 0.856 | 0.852 | 3 min 18s |
| L2 | 3747 | 0.927 | 0.928 | 0.926 | 3 min 19s |
| KL | 3747 | 0.933 | 0.932 | 0.937 | 3 min 26s |



# with an additional layer

## Micro Table

| Loss | Train Iteration | Precision | Recall | F1-score | Training time |
| --- | --- | --- | --- | --- | --- |
| CE | 3747 | 0.940 | 0.940 | 0.940 | 3 min 38s |
| DLITE | 3747 | 0.850 | 0.850 | 0.850 | 3 min 36s |
| L1 | 3747 | 0.890 | 0.890 | 0.890 | 3 min 29s |
| L2 | 3747 | 0.927 | 0.927 | 0.927 | 3 min 29s |
| KL | 3747 | 0.928 | 0.928 | 0.928 | 3 min 42s |


## Macro Table

| Loss | Train Iteration | Precision | Recall | F1-score | Training time |
| --- | --- | --- | --- | --- | --- |
| CE | 3747 | 0.721 | 0.676 | 0.693 | 3 min 38s |
| DLITE | 3747 | 0.983 | 0.111 | 0.102 | 3 min 36s |
| L1 | 3747 | 0.612 | 0.309 | 0.331 | 3 min 29s |
| L2 | 3747 | 0.688 | 0.585 | 0.611 | 3 min 29s |
| KL | 3747 | 0.675 | 0.615 | 0.636 | 3 min 42s |

## Weighted Table

| Loss | Train Iteration | Precision | Recall | F1-score | Training time |
| --- | --- | --- | --- | --- | --- |
| CE | 3747 | 0.938 | 0.940 | 0.938 | 3 min 38s |
| DLITE | 3747 | 0.872 | 0.850 | 0.781 | 3 min 36s |
| L1 | 3747 | 0.860 | 0.890 | 0.867 | 3 min 29s |
| L2 | 3747 | 0.922 | 0.927 | 0.923 | 3 min 29s |
| KL | 3747 | 0.925 | 0.928 | 0.925 | 3 min 42s |


L1 and CE LOSS are slightly better from overall outcome
L2 and KL are slightly worse, probably because of the logistical layer?
DLITE has no overall change, probably still bug even on different models.

In [ ]:
%%bash
DATADIR="ner_english"
if test ! -d "$DATADIR";then
    echo "Creating $DATADIR dir"
    mkdir "$DATADIR"
    cd "$DATADIR"
    wget https://raw.githubusercontent.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL/master/data/train.txt
    wget https://raw.githubusercontent.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL/master/data/test.txt
    wget https://raw.githubusercontent.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL/master/data/dev.txt
fi

Creating ner_english dir


--2024-03-08 19:58:17--  https://raw.githubusercontent.com/mxhofer/Named-Entity-Recognition-BidirectionalLSTM-CNN-CoNLL/master/data/train.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3283420 (3.1M) [text/plain]
Saving to: ‘train.txt’

     0K .......... .......... .......... .......... ..........  1% 4.06M 1s
    50K .......... .......... .......... .......... ..........  3% 5.15M 1s
   100K .......... .......... .......... .......... ..........  4% 13.4M 1s
   150K .......... .......... .......... .......... ..........  6% 42.3M 0s
   200K .......... .......... .......... .......... ..........  7% 7.40M 0s
   250K .......... .......... .......... .......... ..........  9% 46.2M 0s
   300K .......... .......... .......... .......... .......... 10% 37.

# Data Preview

In [ ]:
"""
Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens
"""
import os
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

sys.path.append("../")


def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

    # throw out -DOCSTART- lines
    #lines = [line for line in lines if not line.startswith("-DOCSTART-")]

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)

    return df

DATADIR = "./ner_english/"

def get_conll2003_data(trainfile=DATADIR + "train.txt",
                  devfile=DATADIR + "dev.txt",
                  testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile)
    print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    dev = read_CoNLL2003_format(devfile)
    print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

    test = read_CoNLL2003_format(testfile)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, dev, test

train, dev, test = get_conll2003_data()

X_train, y_train = train.tokens, train.labels
X_dev, y_dev = dev.tokens, dev.labels
X_test, y_test = test.tokens, test.labels


label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)

Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens

NER tags: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']


In [ ]:
label2id = {k: v for v, k in enumerate(label_list)}
id2label = {v: k for v, k in enumerate(label_list)}
label2id

{'B-LOC': 0,
 'B-MISC': 1,
 'B-ORG': 2,
 'B-PER': 3,
 'I-LOC': 4,
 'I-MISC': 5,
 'I-ORG': 6,
 'I-PER': 7,
 'O': 8}

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig, AutoModel,AutoConfig

In [ ]:
from transformers import BertTokenizer, BertConfig, BertForTokenClassification,AutoTokenizer,AutoModelForTokenClassification

In [ ]:
import torch.nn as nn
from transformers import BertModel

class CustomBERTModel(nn.Module):
    def __init__(self):
          super(CustomBERTModel, self).__init__()
          self.bert = BertModel.from_pretrained("bert-base-uncased")
          # add your additional layers here, for example a dropout layer followed by a linear classification head
          self.dropout = nn.Dropout(0.3)
          self.out = nn.Linear(768, 2)

    def forward(self, ids, mask, token_type_ids):
          sequence_output, pooled_output = self.bert(
               ids,
               attention_mask=mask,
               token_type_ids=token_type_ids
          )

          # we apply dropout to the sequence output, tensor has shape (batch_size, sequence_length, 768)
          sequence_output = self.dropout(sequence_output)

          # next, we apply the linear layer. The linear layer (which applies a linear transformation)
          # takes as input the hidden states of all tokens (so seq_len times a vector of size 768, each corresponding to
          # a single token in the input sequence) and outputs 2 numbers (scores, or logits) for every token
          # so the logits are of shape (batch_size, sequence_length, 2)
          logits = self.out(sequence_output)

          return logits

In [ ]:
class MyTaskSpecificCustomModel(nn.Module):
    """
    A task-specific custom transformer model. This model loads a pre-trained transformer model and adds a new dropout
    and linear layer at the end for fine-tuning and prediction on specific tasks.
    """
    def __init__(self, checkpoint, num_labels ):
        """
        Args:
            checkpoint (str): The name of the pre-trained model or path to the model weights.
            num_labels (int): The number of output labels in the final classification layer.
        """
        super(MyTaskSpecificCustomModel, self).__init__()
        self.num_labels = num_labels

        self.model = model = AutoModel.from_pretrained(checkpoint, config = AutoConfig.from_pretrained(checkpoint,
                                                                                                       output_attention = True,
                                                                                                       output_hidden_state = True))
        # New Layer
        self.dropout = nn.Dropout(0.1)
        self.classifier = nn.Linear(768, num_labels)

    def forward(self, input_ids = None, attention_mask=None, labels = None):
        """
        Forward pass for the model.

        Args:
            input_ids (torch.Tensor, optional): Tensor of input IDs. Defaults to None.
            attention_mask (torch.Tensor, optional): Tensor for attention masks. Defaults to None.
            labels (torch.Tensor, optional): Tensor for labels. Defaults to None.

        Returns:
            TokenClassifierOutput: A named tuple with the following fields:
            - loss (torch.FloatTensor of shape (1,), optional, returned when label_ids is provided) – Classification loss.
            - logits (torch.FloatTensor of shape (batch_size, num_labels)) – Classification scores before SoftMax.
            - hidden_states (tuple(torch.FloatTensor), optional, returned when output_hidden_states=True is passed or when config.output_hidden_states=True) – Tuple of torch.FloatTensor (one for the output of the embeddings + one for the output of each layer) of shape (batch_size, sequence_length, hidden_size).
            - attentions (tuple(torch.FloatTensor), optional, returned when output_attentions=True is passed or when config.output_attentions=True) – Tuple of torch.FloatTensor (one for each layer) of shape (batch_size, num_heads, sequence_length, sequence_length).
        """
        outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)

        last_hidden_state = outputs[0]

        sequence_outputs = self.dropouts(last_hidden_state)

        logits = self.classifier(sequence_outputs[:, 0, : ].view(-1, 768 ))

        loss = None
        loss = None
        if labels is not None:
            loss_func = nn.CrossEntropyLoss()
            loss = loss_func(logits.view(-1, self.num_labels), labels.view(-1))

            return logits



In [ ]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.nn import CrossEntropyLoss
from transformers import BertForTokenClassification

class CoNLLClassifier(BertForTokenClassification):
    def __init__(self, config):
        super(CoNLLClassifier, self).__init__(config)
        # Assuming `config.hidden_size` is the size of the hidden layers in the BERT model
        # Define an additional dense layer
        self.additional_layer = nn.Linear(config.hidden_size, config.hidden_size)
        # Define ReLU activation function
        self.activation_fn = nn.ReLU()
        # It's also common to include dropout for regularization
        self.additional_dropout = nn.Dropout(config.hidden_dropout_prob)
        # Ensure the classifier layer is defined correctly following the added layer
        self.classifier = nn.Linear(config.hidden_size, config.num_labels)

    def forward(self, input_ids, attention_mask=None, token_type_ids=None,
                position_ids=None, head_mask=None, labels=None, label_masks=None):
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask)

        sequence_output = outputs[0]  # (b, MAX_LEN, 768)
        token_reprs = [embedding[mask] for mask, embedding in zip(label_masks, sequence_output)]
        token_reprs = pad_sequence(sequences=token_reprs, batch_first=True, padding_value=-1)  # (b, local_max_len, 768)

        # Apply the additional layer and activation function
        sequence_output = self.additional_layer(token_reprs)
        sequence_output = self.activation_fn(sequence_output)
        sequence_output = self.additional_dropout(sequence_output)

        # Dropout and classifier layer remain unchanged
        logits = self.classifier(sequence_output)  # (b, local_max_len, num_labels)
        outputs = (logits,)
        if labels is not None:
            labels = [label[mask] for mask, label in zip(label_masks, labels)]
            labels = pad_sequence(labels, batch_first=True, padding_value=-1)  # (b, local_max_len)
            loss_fct = CrossEntropyLoss(ignore_index=-1, reduction='sum')
            mask = labels != -1
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            loss /= mask.float().sum()
            outputs = (loss,) + outputs + (labels, )

        return outputs


# Data preparing

In [ ]:
MAX_LEN = 173
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenization (it is not needed in this case)

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

# Bert imput

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        tokens = self.data.tokens[index]
        labels = self.data.labels[index]
        tokenized_sentence, labels = tokens, labels

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

# define training size 80/20 split


train_dataset = train
test_dataset = dev


print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

# pyTorch dataloaders

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


# Define the model

model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id,
                                                   output_hidden_states=True)


model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(id2label),
    id2label=id2label,
    label2id=label2id,
    output_attentions = False,
    output_hidden_states = False)

model.to(device)
# Train the model


ids = training_set[0]["ids"].unsqueeze(1)
mask = training_set[0]["mask"].unsqueeze(1)
targets = training_set[0]["targets"].unsqueeze(1)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

TRAIN Dataset: (14987, 2)
TEST Dataset: (3466, 2)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
!pip install activations

In [ ]:
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

TRAIN Dataset: (14987, 2)
TEST Dataset: (3466, 2)


In [ ]:
print(ids.shape)
print(mask.shape)
print(targets.shape)

torch.Size([173, 1])
torch.Size([173, 1])
torch.Size([173, 1])


In [ ]:
ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

tensor(2.5892, device='cuda:0', grad_fn=<NllLossBackward0>)

In [ ]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 173, 9])

# Training set

In [ ]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Training steps: {nb_tr_steps}")

    tr_loss = []


In [ ]:
%%time
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training loss per 100 training steps: 2.4579179286956787
Training loss per 100 training steps: 0.10002840675487376
Training loss per 100 training steps: 0.07105343869947527
Training loss per 100 training steps: 0.059391982497591714
Training loss per 100 training steps: 0.05136037542614761
Training loss per 100 training steps: 0.046543968673200604
Training loss per 100 training steps: 0.043137705006238425
Training loss per 100 training steps: 0.04020655651914936
Training loss per 100 training steps: 0.03798564405255508
Training loss per 100 training steps: 0.036379697325367576
Training loss per 100 training steps: 0.035105001386399236
Training loss per 100 training steps: 0.03361987866353215
Training loss per 100 training steps: 0.03240862152683428
Training loss per 100 training steps: 0.0313416259349603
Training loss per 100 training steps: 0.030689103183641347
Training loss per 100 training steps: 0.02985696064688012
Training loss per 100 training steps: 0.0293964169

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy
            # compute the cohen kappa score

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]


    #print(labels)
    #print(predictions)

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    print(f"Validation steps: {nb_eval_steps}")

    return labels, predictions

In [ ]:
%%time
labels, predictions = valid(model, testing_loader)

Validation loss per 100 evaluation steps: 0.022053580731153488
Validation loss per 100 evaluation steps: 0.017657223807594227
Validation loss per 100 evaluation steps: 0.01684901797060191
Validation loss per 100 evaluation steps: 0.017462209322654364
Validation loss per 100 evaluation steps: 0.017983776635641754
Validation loss per 100 evaluation steps: 0.0181743887582281
Validation loss per 100 evaluation steps: 0.017921910232597687
Validation loss per 100 evaluation steps: 0.017748743202701294
Validation loss per 100 evaluation steps: 0.017825204064437994
Validation loss per 100 evaluation steps: 0.01793540854285737
Validation loss per 100 evaluation steps: 0.01790176626689192
Validation loss per 100 evaluation steps: 0.01778188542069336
Validation loss per 100 evaluation steps: 0.018083857660303276
Validation loss per 100 evaluation steps: 0.018173147083980355
Validation loss per 100 evaluation steps: 0.018145490790762425
Validation loss per 100 evaluation steps: 0.01817946356406451

In [ ]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 832.8 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=70c10885d4d48f6bcb702dfd779510c4508e6819234247b9ca2fc85e1712294d
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

       B-LOC       0.75      0.70      0.72      1837
      B-MISC       0.59      0.37      0.46       922
       B-ORG       0.65      0.73      0.69      1341
       B-PER       0.84      0.78      0.81      1842
       I-LOC       0.55      0.56      0.56       257
      I-MISC       0.66      0.43      0.52       346
       I-ORG       0.62      0.69      0.65       751
       I-PER       0.86      0.85      0.86      1307
           O       0.97      0.98      0.98     49907

    accuracy                           0.94     58510
   macro avg       0.72      0.68      0.69     58510
weighted avg       0.94      0.94      0.94     58510



In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, predictions, average = "micro")

(0.9401640745171765, 0.9401640745171765, 0.9401640745171765, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "macro")

(0.7219840822429133, 0.6764159641276228, 0.6931664964853202, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "weighted")

(0.9378319115964262, 0.9401640745171765, 0.938309440579291, None)

In [ ]:
# L1 loss function

In [ ]:
"""
Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens
"""
import os
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

sys.path.append("../")

def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

    # throw out -DOCSTART- lines
    #lines = [line for line in lines if not line.startswith("-DOCSTART-")]

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)

    return df


DATADIR = "./ner_english/"

def get_conll2003_data(trainfile=DATADIR + "train.txt",
                  devfile=DATADIR + "dev.txt",
                  testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile)
    print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    dev = read_CoNLL2003_format(devfile)
    print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

    test = read_CoNLL2003_format(testfile)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, dev, test

train, dev, test = get_conll2003_data()

X_train, y_train = train.tokens, train.labels
X_dev, y_dev = dev.tokens, dev.labels
X_test, y_test = test.tokens, test.labels


label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)

Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens

NER tags: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenization (it is not needed in this case)

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

# Bert imput

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        tokens = self.data.tokens[index]
        labels = self.data.labels[index]
        tokenized_sentence, labels = tokens, labels

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

# define training size 80/20 split


train_dataset = train
test_dataset = test


print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

# pyTorch dataloaders

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


# Define the model
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id,
                                                   output_hidden_states=True)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(id2label),
    id2label=id2label,
    label2id=label2id,
    output_attentions = False,
    output_hidden_states = False)


model.to(device)

# Train the model

ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

TRAIN Dataset: (14987, 2)
TEST Dataset: (3684, 2)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def one_hot_encode(targets, num_classes):
    """
    One-hot encode a tensor of class indices.

    Arguments:
    - targets (torch.Tensor): A tensor of class indices, shape [N].
    - num_classes (int): The number of classes.

    Returns:
    - torch.Tensor: The one-hot encoded tensor, shape [N, num_classes].
    """
    return F.one_hot(targets, num_classes=num_classes).float()

In [ ]:
l1_loss_fn = nn.L1Loss()

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits

        # implement l1 loss function
        flatten_targets = targets.view(-1) # shape (batch_size * seq_len,)
        flatten_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)

        targets_one_hot = one_hot_encode(flatten_targets, num_classes= flatten_logits.size(-1))
        loss = l1_loss_fn(flatten_logits, targets_one_hot)


        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training l1 loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Training steps: {nb_tr_steps}")

In [ ]:
%%time
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training l1 loss per 100 training steps: 0.3400125503540039
Training l1 loss per 100 training steps: 0.055244903292248744
Training l1 loss per 100 training steps: 0.04365289834008288
Training l1 loss per 100 training steps: 0.03844434545805486
Training l1 loss per 100 training steps: 0.03531135969384204
Training l1 loss per 100 training steps: 0.033025478126254386
Training l1 loss per 100 training steps: 0.03109907973154809
Training l1 loss per 100 training steps: 0.029583324664956577
Training l1 loss per 100 training steps: 0.028233844932219882
Training l1 loss per 100 training steps: 0.02707116606104685
Training l1 loss per 100 training steps: 0.026012961338688207
Training l1 loss per 100 training steps: 0.025061648673158847
Training l1 loss per 100 training steps: 0.024196867319250435
Training l1 loss per 100 training steps: 0.023406282762390526
Training l1 loss per 100 training steps: 0.022697870099091428
Training l1 loss per 100 training steps: 0.0220613252274756

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits


            # implement l1 loss
            flatten_targets = targets.view(-1)
            flatten_logits = eval_logits.view(-1, model.num_labels)
            targets_one_hot = one_hot_encode(flatten_targets, num_classes= flatten_logits.size(-1))
            loss = l1_loss_fn(flatten_logits, targets_one_hot)

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation l1 loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)




    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    print(f"Validation steps: {nb_eval_steps}")

    return labels, predictions

In [ ]:
%%time
labels, predictions = valid(model, testing_loader)

Validation l1 loss per 100 evaluation steps: 0.0048906696029007435
Validation l1 loss per 100 evaluation steps: 0.0055676527173802405
Validation l1 loss per 100 evaluation steps: 0.00527084444820955
Validation l1 loss per 100 evaluation steps: 0.005433568789738159
Validation l1 loss per 100 evaluation steps: 0.005476555027569321
Validation l1 loss per 100 evaluation steps: 0.005434722074041675
Validation l1 loss per 100 evaluation steps: 0.005386427089032685
Validation l1 loss per 100 evaluation steps: 0.005399581895856859
Validation l1 loss per 100 evaluation steps: 0.005367764547064937
Validation l1 loss per 100 evaluation steps: 0.0053526789294173335
Validation l1 loss per 100 evaluation steps: 0.005368904595576845
Validation l1 loss per 100 evaluation steps: 0.005362114280108684
Validation l1 loss per 100 evaluation steps: 0.005366984407396539
Validation l1 loss per 100 evaluation steps: 0.005366057734134722
Validation l1 loss per 100 evaluation steps: 0.005357419126344676
Validati

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-LOC       0.61      0.38      0.47      1668
      B-MISC       0.33      0.03      0.05       702
       B-ORG       0.50      0.40      0.45      1661
       B-PER       0.55      0.45      0.49      1617
       I-LOC       0.00      0.00      0.00       257
      I-MISC       0.00      0.00      0.00       216
       I-ORG       0.00      0.00      0.00       835
       I-PER       0.61      0.54      0.57      1156
           O       0.92      0.99      0.95     45922

    accuracy                           0.89     54034
   macro avg       0.39      0.31      0.33     54034
weighted avg       0.85      0.89      0.87     54034



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, predictions, average = "micro", zero_division = 1)

(0.8901432431432061, 0.8901432431432061, 0.8901432431432061, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "macro", zero_division = 1)

(0.6128010074298055, 0.30990451577317385, 0.3317289836825475, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "weighted", zero_division = 1)

(0.8605589018231765, 0.8901432431432061, 0.8671435576149141, None)

In [ ]:
# L2 function

In [ ]:
"""
Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens
"""
import os
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

sys.path.append("../")


def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

    # throw out -DOCSTART- lines
    #lines = [line for line in lines if not line.startswith("-DOCSTART-")]

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)

    return df

DATADIR = "./ner_english/"

def get_conll2003_data(trainfile=DATADIR + "train.txt",
                  devfile=DATADIR + "dev.txt",
                  testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile)
    print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    dev = read_CoNLL2003_format(devfile)
    print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

    test = read_CoNLL2003_format(testfile)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, dev, test

train, dev, test = get_conll2003_data()

X_train, y_train = train.tokens, train.labels
X_dev, y_dev = dev.tokens, dev.labels
X_test, y_test = test.tokens, test.labels


label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)

Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens

NER tags: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenization (it is not needed in this case)

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence.strip()

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

# Bert imput

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        tokens = self.data.tokens[index]
        labels = self.data.labels[index]
        tokenized_sentence, labels = tokens, labels

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

# define training size 80/20 split


train_dataset = train
test_dataset = test


print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

# pyTorch dataloaders

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


# Define the model
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id,
                                                   output_hidden_states=True)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(id2label),
    id2label=id2label,
    label2id=label2id,
    output_attentions = False,
    output_hidden_states = False)

model.to(device)

# Train the model

ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)



TRAIN Dataset: (14987, 2)
TEST Dataset: (3684, 2)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def one_hot_encode(targets, num_classes):
    """
    One-hot encode a tensor of class indices.

    Arguments:
    - targets (torch.Tensor): A tensor of class indices, shape [N].
    - num_classes (int): The number of classes.

    Returns:
    - torch.Tensor: The one-hot encoded tensor, shape [N, num_classes].
    """
    return F.one_hot(targets, num_classes=num_classes).float()

In [ ]:
l2_loss_fn = nn.MSELoss()

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()

    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits

        # implement l2 loss function
        flatten_targets = targets.view(-1) # shape (batch_size * seq_len,)
        flatten_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)

        targets_one_hot = one_hot_encode(flatten_targets, num_classes= flatten_logits.size(-1))
        loss = l2_loss_fn(flatten_logits, targets_one_hot)


        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training l1 loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Training steps: {nb_tr_steps}")

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training l1 loss per 100 training steps: 0.22480149567127228
Training l1 loss per 100 training steps: 0.012046752177123534
Training l1 loss per 100 training steps: 0.008279891093887055
Training l1 loss per 100 training steps: 0.006842876340887599
Training l1 loss per 100 training steps: 0.006024195501137237
Training l1 loss per 100 training steps: 0.005438634885025179
Training l1 loss per 100 training steps: 0.005049751479544378
Training l1 loss per 100 training steps: 0.004707832827356335
Training l1 loss per 100 training steps: 0.004426759258714899
Training l1 loss per 100 training steps: 0.004183447655493478
Training l1 loss per 100 training steps: 0.004002425583664674
Training l1 loss per 100 training steps: 0.0038261556386529763
Training l1 loss per 100 training steps: 0.003681079719388558
Training l1 loss per 100 training steps: 0.003559566707950555
Training l1 loss per 100 training steps: 0.003447711059125971
Training l1 loss per 100 training steps: 0.003336768

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits


            # implement l2 loss
            flatten_targets = targets.view(-1)
            flatten_logits = eval_logits.view(-1, model.num_labels)
            targets_one_hot = one_hot_encode(flatten_targets, num_classes= flatten_logits.size(-1))
            loss = l2_loss_fn(flatten_logits, targets_one_hot)

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation l1 loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)


    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    print(f"Validation steps: {nb_eval_steps}")

    return labels, predictions

In [ ]:
labels, predictions = valid(model, testing_loader)

Validation l1 loss per 100 evaluation steps: 0.0018311975290998816
Validation l1 loss per 100 evaluation steps: 0.0012351325423500528
Validation l1 loss per 100 evaluation steps: 0.00132480836874488
Validation l1 loss per 100 evaluation steps: 0.0013538042142218756
Validation l1 loss per 100 evaluation steps: 0.001457899386388223
Validation l1 loss per 100 evaluation steps: 0.001477457344935284
Validation l1 loss per 100 evaluation steps: 0.0014696420825106098
Validation l1 loss per 100 evaluation steps: 0.0015116342482764536
Validation l1 loss per 100 evaluation steps: 0.0014852633689764624
Validation l1 loss per 100 evaluation steps: 0.0014870512753417668
Validation l1 loss per 100 evaluation steps: 0.001487928490623072
Validation l1 loss per 100 evaluation steps: 0.001492886331933064
Validation l1 loss per 100 evaluation steps: 0.001474271961981456
Validation l1 loss per 100 evaluation steps: 0.001487956603923773
Validation l1 loss per 100 evaluation steps: 0.0014790275137802368
Val

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

       B-LOC       0.69      0.59      0.64      1668
      B-MISC       0.53      0.32      0.40       702
       B-ORG       0.73      0.55      0.63      1661
       B-PER       0.72      0.80      0.76      1617
       I-LOC       0.45      0.33      0.38       257
      I-MISC       0.72      0.17      0.27       216
       I-ORG       0.58      0.66      0.62       835
       I-PER       0.80      0.87      0.83      1156
           O       0.96      0.98      0.97     45922

    accuracy                           0.93     54034
   macro avg       0.69      0.59      0.61     54034
weighted avg       0.92      0.93      0.92     54034



In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, predictions, average = "micro", zero_division = 1)

(0.9271569752378133, 0.9271569752378133, 0.9271569752378133, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "macro", zero_division = 1)

(0.6878612320992653, 0.5856072608850558, 0.611148031589909, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "weighted", zero_division = 1)

(0.9224661302709939, 0.9271569752378133, 0.9230027370235004, None)

In [ ]:
#Dlite loss function

In [ ]:
"""
Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens
"""
import os
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

sys.path.append("../")

def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

    # throw out -DOCSTART- lines
    #lines = [line for line in lines if not line.startswith("-DOCSTART-")]

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)

    return df


DATADIR = "./ner_english/"

def get_conll2003_data(trainfile=DATADIR + "train.txt",
                  devfile=DATADIR + "dev.txt",
                  testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile)
    print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    dev = read_CoNLL2003_format(devfile)
    print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

    test = read_CoNLL2003_format(testfile)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, dev, test

train, dev, test = get_conll2003_data()

X_train, y_train = train.tokens, train.labels
X_dev, y_dev = dev.tokens, dev.labels
X_test, y_test = test.tokens, test.labels


label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)

Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens

NER tags: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenization (it is not needed in this case)

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

# Bert imput

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        tokens = self.data.tokens[index]
        labels = self.data.labels[index]
        tokenized_sentence, labels = tokens, labels

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

# define training size 80/20 split


train_dataset = train
test_dataset = test


print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

# pyTorch dataloaders

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


# Define the model
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id,
                                                   output_hidden_states=True)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(id2label),
    id2label=id2label,
    label2id=label2id,
    output_attentions = False,
    output_hidden_states = False)

model.to(device)

# Train the model

ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

TRAIN Dataset: (14987, 2)
TEST Dataset: (3684, 2)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# DLITE Loss function
class DLITELoss(nn.Module):
    def __init__(self):
        super(DLITELoss, self).__init__()

    def forward(self, logits, targets, epsilon=1e-10):
        # Convert logits to probabilities using softmax
        probs = F.softmax(logits, dim=-1)

        # One-hot encode the targets to get true probabilities
        true_probs = F.one_hot(targets, num_classes=probs.size(-1)).float()

        # Define the g function
        g_values = torch.abs(probs * (1 - torch.log(probs + epsilon)) - true_probs * (1 - torch.log(true_probs + epsilon)))

        # Define the delta_h function
        delta_h_values = torch.abs(probs**2 * (1 - 2 * torch.log(probs + epsilon)) - true_probs**2 * (1 - 2 * torch.log(true_probs + epsilon))) / (2 * (probs + true_probs))

        # Compute DLITE loss for each class
        dl_values = g_values - delta_h_values

        # Sum over all classes and average over batch size
        loss = dl_values.sum(dim=-1).mean()

        return loss

In [ ]:
dlite_loss_fn = DLITELoss()

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()


    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits



        # implement Dlite loss function
        flatten_targets = targets.view(-1) # shape (batch_size * seq_len,)
        flatten_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)

        loss = dlite_loss_fn(flatten_logits, flatten_targets)
        tr_loss += loss.item()

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training dlite loss per 100 training steps: {loss_step}")


        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Training steps: {nb_tr_steps}")

In [ ]:
%%time
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training dlite loss per 100 training steps: 0.679557204246521
Training dlite loss per 100 training steps: 0.0333419585978557
Training dlite loss per 100 training steps: 0.025475730960817997
Training dlite loss per 100 training steps: 0.02333164553349819
Training dlite loss per 100 training steps: 0.021747106747054948
Training dlite loss per 100 training steps: 0.02093453503930904
Training dlite loss per 100 training steps: 0.02014573694262173
Training dlite loss per 100 training steps: 0.01974439222759869
Training dlite loss per 100 training steps: 0.019451556662106295
Training dlite loss per 100 training steps: 0.01933756405682759
Training dlite loss per 100 training steps: 0.019103211071068927
Training dlite loss per 100 training steps: 0.019029646172031945
Training dlite loss per 100 training steps: 0.018925614431807708
Training dlite loss per 100 training steps: 0.018903241310735553
Training dlite loss per 100 training steps: 0.018730444270027416
Training dlite lo

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits

            flatten_targets = targets.view(-1) # shape (batch_size * seq_len,)
            flatten_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            # implement dlite loss
            loss = dlite_loss_fn(flatten_logits, flatten_targets)
            eval_loss += loss.item()

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy


            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation dlite loss per 100 evaluation steps: {loss_step}")

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)



    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    print(f"Validation steps: {nb_eval_steps}")

    return labels, predictions

In [ ]:
%%time
labels, predictions = valid(model, testing_loader)

Validation dlite loss per 100 evaluation steps: 0.03906558081507683
Validation dlite loss per 100 evaluation steps: 0.016324678749460964
Validation dlite loss per 100 evaluation steps: 0.016891715581686416
Validation dlite loss per 100 evaluation steps: 0.016653731781258386
Validation dlite loss per 100 evaluation steps: 0.016544182826994342
Validation dlite loss per 100 evaluation steps: 0.017187872051807628
Validation dlite loss per 100 evaluation steps: 0.017383375177928997
Validation dlite loss per 100 evaluation steps: 0.017706984837720707
Validation dlite loss per 100 evaluation steps: 0.01768645588121326
Validation dlite loss per 100 evaluation steps: 0.01741036070125846
Validation dlite loss per 100 evaluation steps: 0.017376738568805443
Validation dlite loss per 100 evaluation steps: 0.01732084131840446
Validation dlite loss per 100 evaluation steps: 0.0173230390674768
Validation dlite loss per 100 evaluation steps: 0.017279862742509863
Validation dlite loss per 100 evaluation

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

       B-LOC       0.00      0.00      0.00      1668
      B-MISC       0.00      0.00      0.00       702
       B-ORG       0.00      0.00      0.00      1661
       B-PER       0.00      0.00      0.00      1617
       I-LOC       0.00      0.00      0.00       257
      I-MISC       0.00      0.00      0.00       216
       I-ORG       0.00      0.00      0.00       835
       I-PER       0.00      0.00      0.00      1156
           O       0.85      1.00      0.92     45922

    accuracy                           0.85     54034
   macro avg       0.09      0.11      0.10     54034
weighted avg       0.72      0.85      0.78     54034



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, predictions, average = "micro", zero_division = 1)

(0.8498723026242736, 0.8498723026242736, 0.8498723026242736, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "macro", zero_division = 1)

(0.9833191447360304, 0.1111111111111111, 0.10209381016536165, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "weighted", zero_division = 1)

(0.8724106281436113, 0.8498723026242736, 0.7809003137602923, None)

In [ ]:
# KL divergence

In [ ]:
"""
Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens
"""
import os
import sys

import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

sys.path.append("../")

def flatten(l):
    return [item for sublist in l for item in sublist]

def read_CoNLL2003_format(filename, idx=3):
    """Read file in CoNLL-2003 shared task format"""

    lines =  open(filename).read().strip()

    # find sentence-like boundaries
    lines = lines.split("\n\n")

    # throw out -DOCSTART- lines
    #lines = [line for line in lines if not line.startswith("-DOCSTART-")]

     # split on newlines
    lines = [line.split("\n") for line in lines]

    # get tokens
    tokens = [[l.split()[0] for l in line] for line in lines]

    # get labels/tags
    labels = [[l.split()[idx] for l in line] for line in lines]

    data= {'tokens': tokens, 'labels': labels}
    df=pd.DataFrame(data=data)

    return df


DATADIR = "./ner_english/"

def get_conll2003_data(trainfile=DATADIR + "train.txt",
                  devfile=DATADIR + "dev.txt",
                  testfile=DATADIR + "test.txt"):

    train = read_CoNLL2003_format(trainfile)
    print("Train data: %d sentences, %d tokens"%(len(train),len(flatten(train.tokens))))

    dev = read_CoNLL2003_format(devfile)
    print("Dev data: %d sentences, %d tokens"%(len(dev),len(flatten(dev.tokens))))

    test = read_CoNLL2003_format(testfile)
    print("Test data: %d sentences, %d tokens"%(len(test),len(flatten(test.tokens))))

    return train, dev, test

train, dev, test = get_conll2003_data()

X_train, y_train = train.tokens, train.labels
X_dev, y_dev = dev.tokens, dev.labels
X_test, y_test = test.tokens, test.labels


label_list = np.unique(flatten(y_train))
label_list = list(label_list)
print("\nNER tags:",label_list)

Train data: 14987 sentences, 204567 tokens
Dev data: 3466 sentences, 51578 tokens
Test data: 3684 sentences, 46666 tokens

NER tags: ['B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER', 'O']


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 4
VALID_BATCH_SIZE = 2
EPOCHS = 1
LEARNING_RATE = 5e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# tokenization (it is not needed in this case)

def tokenize_and_preserve_labels(sentence, text_labels, tokenizer):
    """
    Word piece tokenization makes it difficult to match word labels
    back up with individual word pieces. This function tokenizes each
    word one at a time so that it is easier to preserve the correct
    label for each subword. It is, of course, a bit slower in processing
    time, but it will help our model achieve higher accuracy.
    """

    tokenized_sentence = []
    labels = []

    sentence = sentence

    for word, label in zip(sentence.split(), text_labels.split(",")):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

# Bert imput

class dataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, index):
        # step 1: tokenize (and adapt corresponding labels)
        tokens = self.data.tokens[index]
        labels = self.data.labels[index]
        tokenized_sentence, labels = tokens, labels

        # step 2: add special tokens (and corresponding labels)
        tokenized_sentence = ["[CLS]"] + tokenized_sentence + ["[SEP]"] # add special tokens
        labels.insert(0, "O") # add outside label for [CLS] token
        labels.insert(-1, "O") # add outside label for [SEP] token

        # step 3: truncating/padding
        maxlen = self.max_len

        if (len(tokenized_sentence) > maxlen):
          # truncate
          tokenized_sentence = tokenized_sentence[:maxlen]
          labels = labels[:maxlen]
        else:
          # pad
          tokenized_sentence = tokenized_sentence + ['[PAD]'for _ in range(maxlen - len(tokenized_sentence))]
          labels = labels + ["O" for _ in range(maxlen - len(labels))]

        # step 4: obtain the attention mask
        attn_mask = [1 if tok != '[PAD]' else 0 for tok in tokenized_sentence]

        # step 5: convert tokens to input ids
        ids = self.tokenizer.convert_tokens_to_ids(tokenized_sentence)

        label_ids = [label2id[label] for label in labels]
        # the following line is deprecated
        #label_ids = [label if label != 0 else -100 for label in label_ids]

        return {
              'ids': torch.tensor(ids, dtype=torch.long),
              'mask': torch.tensor(attn_mask, dtype=torch.long),
              #'token_type_ids': torch.tensor(token_ids, dtype=torch.long),
              'targets': torch.tensor(label_ids, dtype=torch.long)
        }

    def __len__(self):
        return self.len

# define training size 80/20 split


train_dataset = train
test_dataset = test


print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

# pyTorch dataloaders

train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)


# Define the model
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels=len(id2label),
                                                   id2label=id2label,
                                                   label2id=label2id,
                                                   output_hidden_states=True)

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels = len(id2label),
    id2label=id2label,
    label2id=label2id,
    output_attentions = False,
    output_hidden_states = False)

model.to(device)

# Train the model

ids = training_set[0]["ids"].unsqueeze(0)
mask = training_set[0]["mask"].unsqueeze(0)
targets = training_set[0]["targets"].unsqueeze(0)
ids = ids.to(device)
mask = mask.to(device)
targets = targets.to(device)
outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
initial_loss = outputs[0]
initial_loss

# optimizer
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

TRAIN Dataset: (14987, 2)
TEST Dataset: (3684, 2)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# KL implementation

In [ ]:
class KLDivergenceLoss(nn.Module):
    def __init__(self, reduction='mean'):
        super(KLDivergenceLoss, self).__init__()
        self.reduction = reduction

    def forward(self, logits, targets):
        # Convert logits to probabilities using softmax
        probs = F.softmax(logits, dim=-1)

        # One-hot encode the targets to get true probabilities
        true_probs = F.one_hot(targets, num_classes=probs.size(-1)).float()

        # Masks for non-zero elements of probs and true_probs
        mask_probs = probs > 0
        mask_true_probs = true_probs > 0

        # Calculate g function for non-zero elements using the mask
        kl_values = torch.zeros_like(probs)
        kl_values[mask_true_probs] = true_probs[mask_true_probs] * torch.log(true_probs[mask_true_probs]/probs[mask_true_probs])

        # Sum over all classes and average over the batch size
        loss = kl_values.sum(dim=-1).mean()

        return loss


In [ ]:
kl_loss_fn = KLDivergenceLoss()

# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()


    for idx, batch in enumerate(training_loader):

        ids = batch['ids'].to(device, dtype = torch.long)
        mask = batch['mask'].to(device, dtype = torch.long)
        targets = batch['targets'].to(device, dtype = torch.long)

        outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
        loss, tr_logits = outputs.loss, outputs.logits

        # implement kL divergence loss function
        flatten_targets = targets.view(-1)
        flatten_logits = tr_logits.view(-1, model.num_labels)

        loss = kl_loss_fn(flatten_logits, flatten_targets)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training KL loss per 100 training steps: {loss_step}")

        # compute training accuracy
        flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
        active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
        targets = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)

        tr_preds.extend(predictions)
        tr_labels.extend(targets)

        tmp_tr_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")
    print(f"Training steps: {nb_tr_steps}")

In [ ]:
%%time
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

Training epoch: 1
Training KL loss per 100 training steps: 2.013615608215332
Training KL loss per 100 training steps: 0.11540140477147433
Training KL loss per 100 training steps: 0.08540455459973853
Training KL loss per 100 training steps: 0.07172085745167296
Training KL loss per 100 training steps: 0.06502350443797628
Training KL loss per 100 training steps: 0.05939675354679485
Training KL loss per 100 training steps: 0.05528480958653778
Training KL loss per 100 training steps: 0.05178971546927943
Training KL loss per 100 training steps: 0.04929394458191523
Training KL loss per 100 training steps: 0.04758490607443242
Training KL loss per 100 training steps: 0.0459145980868042
Training KL loss per 100 training steps: 0.04447095011099564
Training KL loss per 100 training steps: 0.043161617073003235
Training KL loss per 100 training steps: 0.042250845044225646
Training KL loss per 100 training steps: 0.0409168250088332
Training KL loss per 100 training steps: 0.04010391786319454
Training

In [ ]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []

    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):

            ids = batch['ids'].to(device, dtype = torch.long)
            mask = batch['mask'].to(device, dtype = torch.long)
            targets = batch['targets'].to(device, dtype = torch.long)

            outputs = model(input_ids=ids, attention_mask=mask, labels=targets)
            loss, eval_logits = outputs.loss, outputs.logits


            # implement KL divergence loss
            flatten_targets = targets.view(-1) # shape (batch_size * seq_len,)
            flatten_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            loss = kl_loss_fn(flatten_logits, flatten_targets)

            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += targets.size(0)

            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation KL loss per 100 evaluation steps: {loss_step}")

            # compute evaluation accuracy
            flattened_targets = targets.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            # now, use mask to determine where we should compare predictions with targets (includes [CLS] and [SEP] token predictions)
            active_accuracy = mask.view(-1) == 1 # active accuracy is also of shape (batch_size * seq_len,)
            targets = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)

            eval_labels.extend(targets)
            eval_preds.extend(predictions)

            tmp_eval_accuracy = accuracy_score(targets.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    #print(eval_labels)
    #print(eval_preds)

    labels = [id2label[id.item()] for id in eval_labels]
    predictions = [id2label[id.item()] for id in eval_preds]

    #print(labels)
    #print(predictions)

    # compute the cohen kappa score

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")
    print(f"Validation steps: {nb_eval_steps}")

    return labels, predictions

In [ ]:
%%time
labels, predictions = valid(model, testing_loader)

Validation KL loss per 100 evaluation steps: 0.08835896104574203
Validation KL loss per 100 evaluation steps: 0.028153014845487958
Validation KL loss per 100 evaluation steps: 0.025673933854686855
Validation KL loss per 100 evaluation steps: 0.02632330794735739
Validation KL loss per 100 evaluation steps: 0.025107370784806383
Validation KL loss per 100 evaluation steps: 0.025932033580020088
Validation KL loss per 100 evaluation steps: 0.02537051426223421
Validation KL loss per 100 evaluation steps: 0.025436891785683476
Validation KL loss per 100 evaluation steps: 0.026302027431353445
Validation KL loss per 100 evaluation steps: 0.02569492581850148
Validation KL loss per 100 evaluation steps: 0.0261218834543202
Validation KL loss per 100 evaluation steps: 0.026134935434045714
Validation KL loss per 100 evaluation steps: 0.025963147337237574
Validation KL loss per 100 evaluation steps: 0.0263162159200684
Validation KL loss per 100 evaluation steps: 0.0259859977914863
Validation KL loss p

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(labels, predictions))

              precision    recall  f1-score   support

       B-LOC       0.77      0.58      0.66      1668
      B-MISC       0.60      0.33      0.42       702
       B-ORG       0.67      0.64      0.66      1661
       B-PER       0.82      0.74      0.78      1617
       I-LOC       0.44      0.39      0.41       257
      I-MISC       0.47      0.38      0.42       216
       I-ORG       0.51      0.70      0.59       835
       I-PER       0.85      0.79      0.82      1156
           O       0.96      0.98      0.97     45922

    accuracy                           0.93     54034
   macro avg       0.68      0.62      0.64     54034
weighted avg       0.93      0.93      0.93     54034



In [ ]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(labels, predictions, average = "micro", zero_division = 1)

(0.9276751674871377, 0.9276751674871377, 0.9276751674871377, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "macro", zero_division = 1)

(0.6755226255369471, 0.6152718068099925, 0.6369761796377479, None)

In [ ]:
precision_recall_fscore_support(labels, predictions, average = "weighted", zero_division = 1)

(0.9250781485378636, 0.9276751674871377, 0.9250553436905857, None)